In [1]:
import pathlib
import os
if not "changed_working_directory" in locals():
    changed_working_directory = True
    os.chdir(pathlib.Path().resolve().parent)
    
%load_ext autoreload
%autoreload 2

In [2]:
from src import constants as const
from utils.timer import load as load_timer
from utils.timer import Timer

import pandas as pd
import numpy as np
import torch
import os
from tqdm.auto import tqdm

imports_path = "src//imports.ipynb"
tabGAN_path = "tabGAN.py"
utils_path = "utils\\utils.ipynb"

from IPython.display import clear_output

In [3]:
timer_path = os.path.join(const.dir.timers(), "timer_tgan.pkl")
if not os.path.exists(timer_path):
    timer = Timer()
    timer.save(timer_path)
    del timer

data_test_tabfairgan = pd.read_csv(os.path.join(const.dir.data(), "df_adult_edited_test.csv"))
data_train_tabfairgan = pd.read_csv(os.path.join(const.dir.data(), "df_adult_edited_train.csv"))

dataset_gen_dir_tabfairgan = os.path.join(const.dir.data_gen(), "tabFairGAN")
dataset_gen_dir_tabfairgan_mod = os.path.join(const.dir.data_gen(), "tabFairGAN-mod")
os.makedirs(dataset_gen_dir_tabfairgan, exist_ok=True)
os.makedirs(dataset_gen_dir_tabfairgan_mod, exist_ok=True)

size_of_fake_data_train = data_train_tabfairgan.shape[0]
size_of_fake_data_test = data_test_tabfairgan.shape[0]

data_train_tabfairgan.columns = data_train_tabfairgan.columns.str.replace(".", "_", regex = False)
#data_test_tabfairgan.columns = data_test_tabfairgan.columns.str.replace(".", "_", regex = False)
path_data_train_tabfairgan = os.path.join(const.dir.data_temp(), "df_adult_edited_train_tabFairGAN.csv")
data_train_tabfairgan.to_csv(path_data_train_tabfairgan)
#data_test_tabfairgan.to_csv(const.dir.data_temp(), "/df_adult_edited_test_tabFairGAN.csv")

n_epochs = 300
batch_size = 500
n_critics = 10
n_synthetic_datasets = 10

dim_hidden_layer = -1 # same as setting equal to input dim
dim_latent_layer = -1 # same as setting equal to input dim
dim_hidden_layer_mod = 256
dim_latent_layer_mod = 128

In [4]:
dataset_gen_path_tabfairgan = os.path.join(dataset_gen_dir_tabfairgan, "gen_train.csv")
timer_tgan = load_timer(timer_path)
timer_tgan.start("tabFairGAN")
%run "src\\tabFairGAN_nofair.py" $path_data_train_tabfairgan $n_epochs $batch_size $dataset_gen_path_tabfairgan --critic_repeat=$n_critics --dim_latent_layer=$dim_latent_layer --dim_hidden_layer=$dim_hidden_layer
timer_tgan.stop("tabFairGAN")
timer_tgan.save(timer_path)
data_gen = pd.read_csv(dataset_gen_path_tabfairgan)
data_gen.columns = data_gen.columns.str.replace("_", ".")
data_gen.to_csv(dataset_gen_path_tabfairgan, index=True)

  0%|          | 0/300 [00:00<?, ?it/s]

Elapsed time for tabFairGAN: 749.747 seconds


In [5]:
timer_tgan = load_timer(timer_path)
timer_tgan.start("tabFairGAN-mod")
dataset_gen_path_tabfairgan_mod = os.path.join(dataset_gen_dir_tabfairgan_mod, "gen_train.csv")
%run "src\\tabFairGAN_nofair.py" $path_data_train_tabfairgan $n_epochs $batch_size $dataset_gen_path_tabfairgan_mod --critic_repeat=$n_critics --dim_latent_layer=$dim_latent_layer_mod --dim_hidden_layer=$dim_hidden_layer_mod 
#--extra_hidden_layer_generator=True
timer_tgan.stop("tabFairGAN-mod")
timer_tgan.save(timer_path)
#clear_output()
data_gen_mod = pd.read_csv(dataset_gen_path_tabfairgan_mod)
data_gen_mod.columns = data_gen_mod.columns.str.replace("_", ".")
data_gen_mod.to_csv(dataset_gen_path_tabfairgan_mod, index=True)

  0%|          | 0/300 [00:00<?, ?it/s]

Elapsed time for tabFairGAN-mod: 751.537 seconds


In [ ]:
#df_name=".//Datasets//adult_edited_train.csv" num_epochs=20 batch_size=500 fake_name=dataset_gen_path_tabfairgan size_of_fake_data=data_train.shape[0]
for i in tqdm(range(n_synthetic_datasets), desc = "Generated datasets", disable = False):
    dataset_train_gen_path_tabfairgan = os.path.join(dataset_gen_dir_tabfairgan, f"gen{i}.csv")
    %run "src\\tabFairGAN_nofair.py" $path_data_train_tabfairgan $n_epochs $batch_size $dataset_train_gen_path_tabfairgan --critic_repeat=$n_critics --dim_hidden_layer=$dim_hidden_layer
    data_train_gen = pd.read_csv(dataset_train_gen_path_tabfairgan)
    data_train_gen.columns = data_train_gen.columns.str.replace("_", ".")
    data_train_gen.to_csv(dataset_train_gen_path_tabfairgan, index=True)

Generated datasets:   0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 0/300 [00:00<?, ?it/s]

In [6]:
for i in tqdm(range(n_synthetic_datasets), desc = "Generated datasets", disable = False):
    dataset_train_gen_path_tabfairgan_mod = os.path.join(dataset_gen_dir_tabfairgan_mod, f"gen{i}.csv")
    %run tabFairGAN_nofair.py $path_ $n_epochs $batch_size $dataset_train_gen_path_tabfairgan_mod --critic_repeat=$n_critics --dim_latent_layer=$dim_latent_layer_mod --dim_hidden_layer=$dim_hidden_layer_mod 
    data_train_gen_mod = pd.read_csv(dataset_train_gen_path_tabfairgan_mod)
    data_train_gen_mod.columns = data_train_gen_mod.columns.str.replace("_", ".")
    data_train_gen_mod.to_csv(dataset_train_gen_path_tabfairgan_mod, index=True)

Generated datasets:   0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

        age         workclass      fnlwgt     education  educational_num  \
0      28.0           Private   150122.53       HS-grad              9.0   
1      44.0  Self-emp-not-inc    82929.12       HS-grad              9.0   
2      34.0        Government    89181.90       Masters             16.0   
3      46.0           Private   229944.98       HS-grad              9.0   
4      49.0        Government   242753.66       HS-grad              9.0   
...     ...               ...         ...           ...              ...   
30157  30.0           Private    13769.00     Bachelors             11.0   
30158  44.0        Government   177555.60  Some-college             11.0   
30159  36.0        Government   203074.58    Assoc-acdm             10.0   
30160  17.0           Private  1484705.00       HS-grad             10.0   
30161  41.0           Private   195481.28       HS-grad             10.0   

      marital_status         occupation   race  gender  capital_gain  \
0      Never-ma

  0%|          | 0/40 [00:00<?, ?it/s]

             age         workclass      fnlwgt     education  educational_num  \
0      35.000000  Self-emp-not-inc  153955.300       Masters             13.0   
1      34.000000           Private  159933.440   Prof-school             13.0   
2      39.000000           Private  150585.020       HS-grad              9.0   
3      33.000000  Self-emp-not-inc  169958.660       HS-grad              9.0   
4      47.000000           Private  255389.750     Bachelors             16.0   
...          ...               ...         ...           ...              ...   
30157  38.000000        Government   64409.040  Some-college             10.0   
30158  29.425226           Private   28173.201     Bachelors             13.0   
30159  25.000000        Government  133088.980     Bachelors             13.0   
30160  50.000000           Private   35732.195     Bachelors             16.0   
30161  37.000000           Private   32876.582     Bachelors             16.0   

      marital_status       

...

        age         workclass       fnlwgt     education  educational_num  \
0      70.0  Self-emp-not-inc   157414.390  Some-college        10.000000   
1      35.0           Private   243149.380  Some-college        11.000000   
2      28.0           Private   120246.984       HS-grad         9.000000   
3      26.0        Government   194315.550     Bachelors        11.509188   
4      52.0           Private    37425.645  Some-college        10.000000   
...     ...               ...          ...           ...              ...   
30157  45.0           Private   309164.340       HS-grad         9.000000   
30158  41.0           Private   188249.080       HS-grad         9.000000   
30159  38.0        Government  1484705.000  Some-college        11.000000   
30160  21.0           Private   177620.730       HS-grad         9.000000   
30161  57.0        Government    46408.750  Some-college        10.000000   

      marital_status         occupation   race  gender  capital_gain  \
0  

  0%|          | 0/40 [00:00<?, ?it/s]

        age     workclass     fnlwgt     education  educational_num  \
0      30.0    Government  105144.84  Some-college             13.0   
1      32.0  Self-emp-inc  258387.42        <=12th              9.0   
2      43.0    Government  176173.88  Some-college             10.0   
3      53.0       Private  170559.83  Some-college             10.0   
4      50.0    Government  115486.38    Assoc-acdm             10.0   
...     ...           ...        ...           ...              ...   
30157  17.0       Private  154102.17       HS-grad              9.0   
30158  31.0    Government  183611.06  Some-college             10.0   
30159  90.0       Private  138933.86        <=12th              5.0   
30160  44.0       Private  231620.66     Bachelors              9.0   
30161  38.0       Private  328577.84     Doctorate             12.0   

      marital_status         occupation                race  gender  \
0      Never-married       Adm-clerical               White    Male   
1    

  0%|          | 0/40 [00:00<?, ?it/s]

        age         workclass     fnlwgt     education  educational_num  \
0      29.0           Private   13769.00       HS-grad             10.0   
1      47.0           Private  192044.06     Assoc-voc             10.0   
2      32.0           Private   13769.00       HS-grad              9.0   
3      17.0           Private  132534.89       HS-grad              9.0   
4      90.0        Government  160665.77       HS-grad              9.0   
...     ...               ...        ...           ...              ...   
30157  30.0           Private  173268.23        <=12th              9.0   
30158  90.0           Private   79756.81       Masters             13.0   
30159  29.0           Private  188901.48  Some-college              9.0   
30160  30.0  Self-emp-not-inc   78971.15       HS-grad             10.0   
30161  21.0           Private  194113.50     Bachelors             10.0   

      marital_status         occupation   race  gender  capital_gain  \
0      Never-married       